In [5]:
from pandas_datareader import data as pdr 
import yfinance as yf
from datetime import datetime
import pandas as pd
import tkinter as tk
import pickle
import import_ipynb
%run error.ipynb

In [6]:
class ydata:
    def __init__(self):
        self._stock=None
        self._interval=None
        self._start=None
        self._end=None
        self._period=None
        self._data=None
        self._industry=None
    
    @property
    def stock(self):
        return self._stock
    @stock.setter
    def stock(self,v):
        self._stock=v
    
    @property
    def interval(self):
        return self._interval
    @interval.setter
    def interval(self,v:str):
        if v in ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']:
            self._interval=v
        else:
            print("Invalid interval")
    
    @property
    def start(self):
        return self._start
    @start.setter
    def start(self,v:str):
        self._start=v
        
    @property
    def end(self):
        return self._end
    @end.setter
    def end(self,v:str):
        self._end=v
    
    @property
    def period(self):
        return self._period
    @period.setter
    def period(self,v:str):
        if v in ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']:
            self._period=v
        else:
            print("Invalid period")
    
    @property
    def data(self):
        return self._data
    @data.setter
    def data(self,v:pd.DataFrame):
        self._data=v
        
    @property
    def industry(self):
        return self._industry
    @industry.setter
    def industry(self,v:pd.Series):
        self._industry=v
        
    def get_data(self,stock:list,interval:str='1d',period:str='1mo',end:str=None,start:str=None):
        if end==None and start==None and period!='max':
            end=datetime.today().strftime('%Y-%m-%d')
        stock_str=''
        for i in stock:
            stock_str+=i+' '
        try:
            yf.pdr_override()
            self._data = pdr.get_data_yahoo(stock_str, group_by = 'ticker', \
                                  interval=interval,start=start,end=end,period=period)
            self.stock=stock
            self.interval=interval
            self.start=self.data.index[0].strftime('%Y-%m-%d')
            self.end=self.data.index[-1].strftime('%Y-%m-%d')
            print('succ')
            return 1
        except:
            print('bad')
            return 0


In [7]:
class download_page(tk.Frame):
    def __init__(self, root, data, stock,interval,start,end,period):
        super().__init__(root)
        
        self.data=data
        self.stock,self.interval,self.start,self.end,self.period=stock,interval,start,end,period
        
        tk.Label(self,text="Download panel data from Yahoo finance",font=("Courier", 44)).place(relx=0.5,rely=0.1,anchor=tk.CENTER)
        
        tk.Label(self,text="stock",font=("Courier", 20)).place(relx=0.2,rely=0.2)
        tk.Entry(self,textvariable=self.stock,width=50).place(relx=0.3,rely=0.2,width=100)
        
        tk.Label(self,text="interval",font=("Courier", 20)).place(relx=0.2,rely=0.25)
        interval_=['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']
        self.interval.set('1d')
        tk.OptionMenu(self,self.interval,*interval_).place(relx=0.3,rely=0.25)
        
        tk.Label(self,text="start",font=("Courier", 20)).place(relx=0.2,rely=0.3)
        tk.Entry(self,textvariable=self.start,width=50).place(relx=0.3,rely=0.3)
        
        tk.Label(self,text="end",font=("Courier", 20)).place(relx=0.2,rely=0.35)
        tk.Entry(self,textvariable=self.end,width=50).place(relx=0.3,rely=0.35)
        
        tk.Label(self,text="period",font=("Courier", 20)).place(relx=0.2,rely=0.4)
        period_=['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']
        self.period.set('1mo')
        tk.OptionMenu(self,self.period,*period_).place(relx=0.3,rely=0.4)
        
        tk.Button(self,text="download",command=self.downloading,font=("Courier", 20)).place(relx=0.33,rely=0.45)
        root.bind('<Return>',self.downloading)
        
    def downloading(self,event=None):
        self.stock,self.interval,self.start,self.end,self.period=\
            self.stock.get().split(),self.interval.get(),self.start.get(),self.end.get(),self.period.get()
        dic_=dict(zip(('stock','interval','start','end','period'),(self.stock,self.interval,self.start,self.end,self.period)))
        dic={}
        for i in dic_:
            if dic_[i]!="":
                dic[i]=dic_[i]
        succ=self.data.get_data(**dic)
        if not succ:
            tk.Label(self,text="Invalid input!").grid(row=8)
        else:
            tk.Label(self,text="success!").grid(row=8)
            self.frame=tk.Frame(self)
            self.table=Table(self.frame,dataframe=self.data.data)
            self.table.show()
            self.frame.grid(row=9)
            try:
                self.data.industry=pd.Series()
                for s in self.stock:
                    self.data.industry[s]=yf.Ticker(s).info['sector']
            except:
                error("Error in accessing industry data")
                
                
            
            

In [8]:
class upload_page(tk.Frame):
    def __init__(self, root, data, address):
        super().__init__(root)
        self.data=data
        self.address=address
        tk.Label(self,text="Upload panel data from this device").grid(row=0)
        tk.Label(self,text="file address").grid(row=1)
        tk.Entry(self,textvariable=self.address).grid(row=2)
        bt=tk.Button(self,text="upload",command=self.uploading)
        bt.grid(row=3)
        
    def uploading(self,event=None):
        try:
            address=self.address.get()
            file=open(address,'rb')
            self.data.data=pickle.load(file)
            tk.Label(self,text="success!").grid(row=4)
            self.frame=tk.Frame(self)
            self.table=Table(self.frame,dataframe=self.data.data)
            self.table.show()
            self.frame.grid(row=5)
        except:
            tk.Label(self,text="Invalid address!").grid(row=4)
        